# t-SVD

An effort to get the tensor SVD up and runnning

In [16]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from random import random, shuffle
import itertools
from numpy.fft import fft, ifft
from scipy.linalg import svd
import matplotlib.pyplot as plt

# Some Previous Work

In [27]:
def printFrontalSlices(A):
    n, m, p = A.shape
    rep = "\n"
    for i in range(0,p):
        #print(A[:,:,i])
        vals = A[:,:,i].reshape(n,m)
        rep += str(vals)
        rep += "\n"
        
    print(rep)

In [10]:
# tensor transpose
def t_tran(A):
    ten_circ = t_circ(A)
    #print(ten_circ.shape)
    ten_circ = ten_circ.transpose()
    #print(ten_circ.shape)
    n2,n1,n3 = A.shape
    #print(n1,n2,n3)
    res = fold_up(ten_circ, n1, n2, n3)
    return res 

# Test Case

In [19]:
test_tensor = np.array([[[1, 2], [2, 3]], [[4, 3], [5, 2]]])
printFrontalSlices(test_tensor)


[[1 2]
 [4 5]]
[[2 3]
 [3 2]]



In [9]:
test_tensor_results = np.array([[[-0.442305, -0.059622], [0.029485, -0.894395]], [[0.029485, -0.894395], [0.442305, 0.059622]]])
printFrontalSlices(test_tensor_results)


[[-0.442305  0.029485]
 [ 0.029485  0.442305]]
[[-0.059622 -0.894395]
 [-0.894395  0.059622]]



In [54]:
# to get my head around some shaping stuff
a = np.array([[[1, 2, 6], [2, 3, 9]], [[4, 3, 9], [5, 2, 0]]])
printFrontalSlices(a)


[[1 2]
 [4 5]]
[[2 3]
 [3 2]]
[[6 9]
 [9 0]]



# Here We Go

In [59]:
def t_svd(A, k = None):
    n1, n2, n3 = A.shape
    
    #print("Shape of input tensor: {} {} {}".format(n1, n2, n3))
    
    # transpose and swap if wrong shape
    if(n2 > n1):
        trans_flag = True
        A = t_tran(A)
        swap = n1
        n1 = n2
        n2 = swap
        
    A = fft(A, axis=2)
    # Correct so far lol
    
    # flag to compute compact or full
    fl = 1
    if k is not None:
        fl = 0
    
    K = min(n1, n2)
    
    if (fl == 0):
        U = np.zeros((n1, K, n3))
        S = np.zeros((K, K, n3))
        V = np.zeros((K, n2, n3))
    
    #print(U.shape)
    #print(S.shape)
    #print(V.shape)
    
    for i in range(0, n3):
        if (fl == 0):
            U1, S1, V1 = svd(A[:,:,i], full_matrices = True)
            U[:,:,i] = U1
            S_mat = np.diag(S1.tolist())
            S[:,:,i] = S_mat
            V[:,:,i] = np.flip(V1)
    
    #printFrontalSlices(U)
    #print("~~~~")
    #for i in range(0, len(S)):
    #    print(S[:,:,i])
    #print("~~~~")
    #printFrontalSlices(V)
    #print("~~~~")
    
    return np.real(ifft(U, axis=2)), np.real(ifft(S, axis=2)), np.real(ifft(V, axis=2))

In [61]:
# Test
U,S,V = t_svd(test_tensor, 1)

printFrontalSlices(U)
printFrontalSlices(S)
printFrontalSlices(V)


[[-0.44230513  0.02948472]
 [ 0.02948472  0.44230513]]
[[-0.05962169 -0.89439481]
 [-0.89439481  0.05962169]]


[[7.4188942  0.        ]
 [0.         0.90566112]]
[[4.00468064 0.        ]
 [0.         0.31987468]]


[[-0.13955957 -0.08710068]
 [ 0.08710068 -0.13955957]]
[[-0.522243    0.83677885]
 [-0.83677885 -0.522243  ]]



/home/tim/Documents/research/env/lib/python3.7/site-packages/ipykernel_launcher.py:36: ComplexWarning: Casting complex values to real discards the imaginary part
/home/tim/Documents/research/env/lib/python3.7/site-packages/ipykernel_launcher.py:39: ComplexWarning: Casting complex values to real discards the imaginary part
